In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv

In [ ]:
html_trial_doc = requests.get("https://www.google.com")
html_trial_doc.status_code

200

In [ ]:
type(html_trial_doc.text)

str

In [ ]:
soup = BeautifulSoup(html_trial_doc.text, 'html.parser')
print(soup.prettify())

In [ ]:
for i in soup.find_all('input'):
    print(i, end='\n')

In [ ]:
!pip install fake_useragent
from fake_useragent import UserAgent

In [ ]:
ua = UserAgent()
html_keyword = "Hello"
google_url = "https://www.google.com/search?q=" + html_keyword + "&num=" + str(20)
response = requests.get(google_url, {"User-Agent": ua.random})
soup = BeautifulSoup(response.text, 'html.parser')

soup.find_all("input")


In [ ]:
r2 = requests.get("https://www.google.com/search?q=Hello&amp;num=20&amp;User-Agent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_10_1%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F41.0.2227.1+Safari%2F537.36")

In [ ]:
soup1 = BeautifulSoup(r2.text, "html.parser")
print(soup1.prettify())

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import re
import time


class GoogleScraper:
    # Crawler entry point
    base_url = 'https://www.google.com/search'

    # Query string parameters to crawl through results pages
    pagination_params = {
        'q': 'linux mint',
        'sxsrf': 'ACYBGNRmhZ3C1fo8pX_gW_d8i4gVeu41Bw:1575654668368',
        'ei': 'DJXqXcmDFumxrgSbnYeQBA',
        'start': '',
        'sa': 'N',
        'ved': '2ahUKEwjJua-Gy6HmAhXpmIsKHZvOAUI4FBDy0wN6BAgMEDI',
        'biw': '811',
        'bih': '628'
    }

    # Query string parameters for initial results page
    initial_params = {
        'sxsrf': 'ACYBGNQ16aJKOqQVdyEW9OtCv8zRsBcRig:1575650951873',
        'source': 'hp',
        'ei': 'h4bqXcT0MuPzqwG87524BQ',
        'q': '',
        'oq': '',
        'gs_l': 'psy-ab.1.1.35i362i39l10.0.0..139811...4.0..0.0.0.......0......gws-wiz.....10.KwbM7vkMEDs'
    }

    # Request headers
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'cookie': 'CGIC=InZ0ZXh0L2h0bWwsYXBwbGljYXRpb24veGh0bWwreG1sLGFwcGxpY2F0aW9uL3htbDtxPTAuOSxpbWFnZS93ZWJwLGltYWdlL2FwbmcsKi8qO3E9MC44LGFwcGxpY2F0aW9uL3NpZ25lZC1leGNoYW5nZTt2PWIz; HSID=AenmNVZxnoADsXz_x; SSID=AjbLhhwkjh8f3FOM8; APISID=IqkNtUA0V2DXlees/A0tA9iPSadMC2X6dt; SAPISID=8-N4B06I_D5N1mvR/AleccT6Zt0QllrukC; CONSENT=YES+UA.en+; OTZ=5204669_48_48_123900_44_436380; SID=rAd3UAFN_dCIGQ87HqDZZGiNyxdz0dL4dZKy_XquqSr_CHTzqSzfDdNTfLmA2xCMEZOZMA.; ANID=AHWqTUnDWUSHdvWhJiIoPxMAKYXmVtHCQIq7LBMYgiSlZZr3AMGTwY2aVUdjeY7z; NID=193=QImFbOa1vnKpflG8yJytqPXbJYJ9k8fWbIzQMGExsMa4g5oJwdnI56WNjgEVFAyAPJ1SEEOQ-zlW4HAUv-JLj0yAUImTgeT1syDIgFTMWAqxdz10lWRlzFC-3Fmjv6xJcqm2o6RKI50dmb7GetiheNdSAYPkAjng_c0lOHoXZLmtMwFOpkPTrQwVyUW8R2x4o1ux3OW3_kEbR_BREowRV8lVqrsnyo1ffC_Pm40zf81k7aS0cv9esYweGHF6Lxd532z4wA; 1P_JAR=2019-12-06-16; DV=k7BRh0-RaJtZsO9g7sjbrkcKoUjC7RYhxDh5AdfYgQAAAID1UoVsAVkvPgAAAFiry7niUB6qLgAAAGCQehpdCXeKnikKAA; SEARCH_SAMESITE=CgQIvI4B; SIDCC=AN0-TYv-lU3aPGmYLEYXlIiyKMnN1ONMCY6B0h_-owB-csTWTLX4_z2srpvyojjwlrwIi1nLdU4',
        'pragma': 'no-cache',
        'referer': 'https://www.google.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/75.0.3770.142 Chrome/75.0.3770.142 Safari/537.36'
    }

    # Scraped results
    results = []

    def fetch(self, query, page):
        '''Makes HTTP GET request to fetch search results from google'''

        # Init initial_params search query (e.g. "linux mint")
        self.initial_params['q'] = query

        # If getting the first results page
        if not page:
            # Use initial params
            params = self.initial_params

        # Otherwise we're scraping the following pages
        else:
            # Use pagination params
            params = self.pagination_params

            # Specify page number in format page * 10
            params['start'] = str(page * 10)

            # Init search query
            params['q'] = query

        # Make HTTP GET request
        response = requests.get(self.base_url, params=params, headers=self.headers)
        print('HTTP GET request to URL: %s | Status code: %s' % (response.url, response.status_code))

        # Return HTTP response
        return response

    def parse(self, html):
        '''Parses response's text and extract data from it'''

        # Parse content
        content = BeautifulSoup(html, 'lxml')

        # Extract data
        title = [title.text for title in content.findAll('span', {'class': 'S3Uucc'})]
        link = [link.next_element['href'] for link in content.findAll('div', {'class': 'r'})]
        description = [descr.text for descr in content.findAll('span', {'class': 'st'})]

        # Loop over the number of entries
        for index in range(0, len(title)):
            # Append extracted data to results list
            self.results.append({
                'title': title[index],
                'link': link[index],
                'description': description[index]
            })

    def write_csv(self):
        '''Writes scpared results to CSV file'''

        # Check results list in not empty
        if len(self.results):
            print('Writing results to "res.csv"... ', end='')

            # Open file stream to write CSV
            with open('res.csv', 'w') as csv_file:
                # Init CSV dictionary writer
                writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())

                # Write column names to file
                writer.writeheader()

                # Write results list to CSV file
                for row in self.results:
                    writer.writerow(row)

            print('Done')

    def store_response(self, response):
        '''Stores HTML response to file for debugging parser'''

        # If response is OK
        if response.status_code == 200:
            print('Saving response to "res.html"... ', end='')

            # Write response to HTML file
            with open('res.html', 'w') as html_file:
                html_file.write(response.text)

            print('Done')
        else:
            print('Bad response!')

    def load_response(self):
        '''Loads HTML response for debugging parser'''
        html = ''

        # Open HTML file
        with open('res.html', 'r') as html_file:
            for line in html_file.read():
                html += line

        # Return HTML as string
        return html

    def run(self):
        '''Starts crawler'''

        # Loop over the range of pages to scrape
        for page in range(0, 5):
            # Make HTTP GET request
            response = self.fetch('linux mint', page)

            # Parse content
            self.parse(response.text)

            # Wait for 5 sec
            time.sleep(5)

        # Write scraped results to CSV file
        self.write_csv()


# Main driver
if __name__ == '__main__':
    scraper = GoogleScraper()
    scraper.run()

HTTP GET request to URL: https://www.google.com/search?sxsrf=ACYBGNQ16aJKOqQVdyEW9OtCv8zRsBcRig%3A1575650951873&source=hp&ei=h4bqXcT0MuPzqwG87524BQ&q=linux+mint&oq=&gs_l=psy-ab.1.1.35i362i39l10.0.0..139811...4.0..0.0.0.......0......gws-wiz.....10.KwbM7vkMEDs | Status code: 200
HTTP GET request to URL: https://www.google.com/search?q=linux+mint&sxsrf=ACYBGNRmhZ3C1fo8pX_gW_d8i4gVeu41Bw%3A1575654668368&ei=DJXqXcmDFumxrgSbnYeQBA&start=10&sa=N&ved=2ahUKEwjJua-Gy6HmAhXpmIsKHZvOAUI4FBDy0wN6BAgMEDI&biw=811&bih=628 | Status code: 200
HTTP GET request to URL: https://www.google.com/search?q=linux+mint&sxsrf=ACYBGNRmhZ3C1fo8pX_gW_d8i4gVeu41Bw%3A1575654668368&ei=DJXqXcmDFumxrgSbnYeQBA&start=20&sa=N&ved=2ahUKEwjJua-Gy6HmAhXpmIsKHZvOAUI4FBDy0wN6BAgMEDI&biw=811&bih=628 | Status code: 200
HTTP GET request to URL: https://www.google.com/search?q=linux+mint&sxsrf=ACYBGNRmhZ3C1fo8pX_gW_d8i4gVeu41Bw%3A1575654668368&ei=DJXqXcmDFumxrgSbnYeQBA&start=30&sa=N&ved=2ahUKEwjJua-Gy6HmAhXpmIsKHZvOAUI4FBDy0wN6BAgMED

In [ ]:
import urllib

class GoogleWebCrawler:

    def __init__(self, query, num = 10, as_epq = '', as_oq = ''):
        if not isinstance(query, str) or not isinstance(as_epq, str) or not isinstance(as_oq, str):
            print("The query is not a string", end="\n")
            return None
        else:
            parsed_query = urllib.parse.quote_plus(query)
            parsed_as_epq = urllib.parse.quote_plus(as_epq)
            parsed_as_oq = urllib.parse.quote_plus(as_oq)

        self.base_url = 'https://www.google.com/search'

        self.parameters = {
            'q': parsed_query,
            'as_epq': parsed_as_epq,
            'as_oq': parsed_as_oq,
            'start': urllib.parse.quote_plus(''),
            'source': 'hp',
            'lr':'en'
        }

        self.request_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'no-cache',
            'pragma': 'no-cache',
            'referer': 'https://www.google.com/',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/75.0.3770.142 Chrome/75.0.3770.142 Safari/537.36'
        }

        self.num = num

        self.results = []


    def fetch(self, query, page_num):

        assert isinstance(page_num, int), "Page not an integer"

        self.parameters['q'] = query
        params = self.parameters
        params['start'] = str(page_num * 10)

        # Make HTTP GET request
        response = requests.get(self.base_url, params=params, headers=self.request_headers)

        print('URL: %s | Status code: %s' % (response.url, response.status_code))

        return response

    def parser(self, response, verbose = False, parser = 'lxml'):
        print(response)
        html_content = BeautifulSoup(response.text, parser)
        if verbose:
            print(html_content.prettify())

        search_results = [html_content.find_all('h3', {'class':'LC20lb DKV0Md'})]
        print(search_results)
        return search_results

    def writer(self):
        pass

    def run(self):

        for page in range(0, self.num):

            response = self.fetch(self.parameters['q'], page)
            #print(response)
            self.parser(response)

            time.sleep(0.5)

if __name__ == '__main__':
    web_crawler = GoogleWebCrawler('linux min')
    web_crawler.run()

In [ ]:
https://appft.uspto.gov/netacgi/nph-Parser?
Sect1=PTO2&
Sect2=HITOFF&
p=1&
u=%2Fnetahtml%2FPTO%2Fsearch-bool.html
&r=0&
f=S&
l=50&
TERM1=carbon&
FIELD1=&
co1=AND&
TERM2=neutral&
FIELD2=&
d=PG01


https://appft.uspto.gov/netacgi/nph-Parser?
Sect1=PTO2&
Sect2=HITOFF&
u=%2Fnetahtml%2FPTO%2Fsearch-adv.html&
r=0&
f=S&
l=50&
d=PG01&
OS=carbon+AND+neutral&
RS=carbon+AND+neutral&
TD=183491&
Srch1=carbon&
Srch2=neutral&
Conj1=AND&
NextList2=Next+50+Hits&
StartNum=&
Query=carbon+AND+neutral


https://appft.uspto.gov/netacgi/nph-Parser?
Sect1=PTO2&
Sect2=HITOFF&
u=%2Fnetahtml%2FPTO%2Fsearch-adv.html&
r=0&
f=S&
l=50&
d=PG01&
OS=carbon+AND+neutral&
RS=carbon+AND+neutral&
TD=183491&
Srch1=carbon&
Srch2=neutral&
Conj1=AND&
NextList3=Next+50+Hits&
StartNum=&
Query=carbon+AND+neutral

https://patft.uspto.gov/netacgi/nph-Parser?
Sect1=PTO2&
Sect2=HITOFF&
p=1&
u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&
r=0&
f=S&
l=50&
TERM1=Florida&
FIELD1=AACI&
co1=OR&
TERM2=Non&
FIELD2=CIPC&
d=PTXT

In [ ]:
class USPTO_Crawler:

    def __init__(self, term_1, term_2):
        if not isinstance(term_1, str) or not isinstance(term_2, str):
            print("The query is not a string", end="\n")
            return None
        else:
            parsed_query = urllib.parse.quote_plus(query)
            parsed_as_epq = urllib.parse.quote_plus(as_epq)
            parsed_as_oq = urllib.parse.quote_plus(as_oq)

        self.base_url = 'https://appft.uspto.gov/netacgi/nph-Parser'

        self.parameters = {
            'q': parsed_query,
            'as_epq': parsed_as_epq,
            'as_oq': parsed_as_oq,
            'start': urllib.parse.quote_plus(''),
            'source': 'hp',
            'lr':'en'
        }

        self.request_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'no-cache',
            'pragma': 'no-cache',
            'referer': 'https://www.google.com/',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/75.0.3770.142 Chrome/75.0.3770.142 Safari/537.36'
        }

        self.num = num

        self.results = []


    def fetch(self, query, page_num):

        assert isinstance(page_num, int), "Page not an integer"

        self.parameters['q'] = query
        params = self.parameters
        params['start'] = str(page_num * 10)

        # Make HTTP GET request
        response = requests.get(self.base_url, params=params, headers=self.request_headers)

        print('URL: %s | Status code: %s' % (response.url, response.status_code))

        return response

    def parser(self, response, verbose = False, parser = 'lxml'):
        print(response)
        html_content = BeautifulSoup(response.text, parser)
        if verbose:
            print(html_content.prettify())

        search_results = [html_content.find_all('h3', {'class':'LC20lb DKV0Md'})]
        print(search_results)
        return search_results

    def writer(self):
        pass

    def run(self):

        for page in range(0, self.num):

            response = self.fetch(self.parameters['q'], page)
            #print(response)
            self.parser(response)

            time.sleep(0.5)

if __name__ == '__main__':


In [ ]:
import json
lst = []
for i in range(2009, 2021):
    if i != 2018 and i != 2019:
        with open(f'{i}.json', 'r') as fp:
            fpr = json.load(fp)
            lst.append(fpr)
## inventionTitle;
for key, value in lst[0].items():
    for corpus in value:
        for key_corpus, value_corpus in corpus.items():
            if isinstance(value_corpus, dict):
                for key_meta, value_meta in value_corpus.items():
                    print(key_meta, value_meta)
        print()

applicationNumberText {'value': 'PCT/US09/03769', 'electronicText': 'PCT/US09/03769'}
filingDate 2009-06-25
applicationTypeCategory PCT
partyBag {'applicantBagOrInventorBagOrOwnerBag': [{'primaryExaminerOrAssistantExaminerOrAuthorizedOfficer': [{'name': {'personNameOrOrganizationNameOrEntityName': [{'personFullName': 'YOUNG, LEE W'}]}}]}, {'applicant': [{'contactOrPublicationContact': [{'name': {'personNameOrOrganizationNameOrEntityName': [{'organizationStandardName': {'content': ['ALIBABA GROUP HOLDING LIMITED']}}]}, 'cityName': 'GEORG TOWN, GRAND CAYMAN', 'geographicRegionName': {'value': '', 'geographicRegionCategory': 'STATE'}, 'countryCode': 'KY'}]}]}, {'inventorOrDeceasedInventor': [{'contactOrPublicationContact': [{'name': {'personNameOrOrganizationNameOrEntityName': [{'personStructuredName': {'firstName': 'TINGJIA', 'middleName': '', 'lastName': 'CHEN'}}]}, 'cityName': 'HANGZHOU', 'geographicRegionName': {'value': '', 'geographicRegionCategory': 'STATE'}, 'countryCode': 'CN'}]}